<a href="https://colab.research.google.com/github/stephenbeckr/convex-optimization-class/blob/master/Demos/ConjugateGradientDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conjugate Gradient
... and related Krylov subspace methods

For solving $Ax=b$ and related problems (e.g., least-squares).  Use Krylov subspace methods if all of the following criteria are met:
1. $A$ is very large (let $A$ be $n\times n$)
2. The multiply $Ax$ can be done faster than $O(n^2)$, e.g.
  - $A$ is very sparse
  - $A$ if from a fast operator, like a FFT
3. $A$ is somewhat well-conditioned

Just how large, or how well-conditioned, or how sparse depends, and there's no simple answer (other than just try it)

APPM 5630 Advanced Convex Optimization, Spring 2023, Becker

In [15]:
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg
import scipy.linalg as linalg
from numpy.linalg import norm
import time

Create a sparse $10\times 10$ matrix

In [16]:
n   = int(1e1)

rng   = np.random.default_rng(1)
# Make sure A is invertrible by adding identity to it
A   = sps.random(n,n,density=0.1,format='csr',random_state=rng) + .1*sps.eye(n)
# A   = sps.random(n,n,density=0.1,format='csr') + .01*sps.eye(n) # worst conditioned, LSQR/CG struggles
b   = rng.normal(size=(n,1))

print(f'condition number is {np.linalg.cond( A.toarray() ):.1f}' )


condition number is 211.6


In [17]:
A

<10x10 sparse matrix of type '<class 'numpy.float64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [18]:
with np.printoptions(precision=3, suppress=True):
  print( A.toarray() )

[[0.1   0.    0.    0.    0.    0.    0.    0.403 0.    0.   ]
 [0.    0.1   0.    0.754 0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.1   0.    0.    0.    0.    0.    0.    0.303]
 [0.788 0.134 0.    0.1   0.538 0.    0.    0.    0.    0.   ]
 [0.    0.    0.262 0.    0.1   0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.1   0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.1   0.    0.    0.   ]
 [0.    0.    0.    0.    0.453 0.    0.    0.1   0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.1   0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.33  0.203 0.1  ]]


In [19]:
x = linalg.solve(A.toarray(),b) # Dense solve
# xCG = sps.linalg.spsolve(A,b)   # Direct solve, taking advantage of sparsity
# xCG, info = sps.linalg.minres(A,b,maxiter=1000) # only if A is symmetric
# xCG = sps.linalg.lsqr(A,b,atol=1e-12,btol=1e-12,iter_lim=int(1e5))[0]
xCG = sps.linalg.lsqr(A,b)[0]
print(f"Discrepancy in the two solutions is {norm(x.ravel()-xCG.ravel()):.2e}")

# If it's ill-conditioned, the two "x" may not be similar, but check
#  that the residual is small:
print(f"Residual ||Ax-b|| for dense solve is {norm(A@x-b):.2e}, and is {norm(A@xCG-b.ravel()):.2e} for LSQR/CG")

Discrepancy in the two solutions is 3.36e-05
Residual ||Ax-b|| for dense solve is 1.24e-15, and is 3.43e-05 for LSQR/CG


In [ ]:
# Be careful with some bugs!
print( norm( b - b) )
print( norm( b - b.ravel() ) )

0.0
16.27126842092771


# Larger example

In [20]:
n   = int(5e3)
rng   = np.random.default_rng(1)
A   = sps.random(n,n,density=0.01,format='csr',random_state=rng) + 10*sps.eye(n)
b   = rng.normal(size=(n,1))

print("Doing dense version")
tic = time.perf_counter()
x = linalg.solve(A.toarray(),b)
toc_dense = time.perf_counter() - tic

print('Now doing sparse version')
tic = time.perf_counter()
# xCG = sps.linalg.spsolve(A,b) # This is very slow! not recommended
xCG = sps.linalg.lsqr(A,b)[0] # nice and fast
toc_sparse = time.perf_counter() - tic

e = norm(x.ravel()-xCG.ravel())
print(f"n x n matrix with n={n:d}")
print(f"Took {toc_dense:.2f} sec for dense version (Gaussian elimination...)")
print(f"Took {toc_sparse:.2f} sec for sparse version (CG, LSQR, ...)")
print(f"Difference between versions {e:.1e}")

Doing dense version
Now doing sparse version
n x n matrix with n=5000
Took 4.76 sec for dense version (Gaussian elimination...)
Took 0.02 sec for sparse version (CG, LSQR, ...)
Difference between versions 8.1e-05
